<a href="https://colab.research.google.com/github/Kiran-01/Projects/blob/master/Diabetes_Prediction_using_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Installing Dependencies
!pip install pyspark

     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 51.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=d009d04106285bdeda18c3e54efcb637c6431862fc95e1f7443285b8a2fd3f34
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [7]:
# Running a SparkSession

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark").getOrCreate()

In [3]:
# uploading the Dataset

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving diabetes.csv to diabetes.csv
User uploaded file "diabetes.csv" with length 62058 bytes


In [8]:
df = spark.read.csv('diabetes.csv',header=True,inferSchema=True)

In [18]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|      0|25.6|                   0.294| 28|      0|


In [11]:
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [9]:
print((df.count(), len(df.columns)))

(2000, 9)


In [10]:
df.groupby('Outcome').count().show()

+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  684|
|      0| 1316|
+-------+-----+



In [12]:
df.describe().show()

+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|    SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|             2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|          121.1825|           69.1455|           20.935|           80.254|32.192999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|32.068635649902916|19.188314815604098|16.103242909926

In [ ]:
#Cleaning the data

In [13]:
# find for null value

for col in df.columns:
  print(col + ":", df[df[col].isNull()].count())

Pregnancies: 0
Glucose: 0
BloodPressure: 0
SkinThickness: 0
Insulin: 0
BMI: 0
DiabetesPedigreeFunction: 0
Age: 0
Outcome: 0


In [14]:
# finding the total number of 0 values in columns : Glucose, Bloodpressure, skinThickness, Insulin and BMI

def count_zeros():
  column_list = ['Glucose', 'BloodPressure','SkinThickness', 'Insulin',  'BMI' ]
  for i in column_list:
    print(i+ ":", df[df[i]==0].count())

          


In [15]:
count_zeros()

Glucose: 13
BloodPressure: 90
SkinThickness: 573
Insulin: 956
BMI: 28


In [19]:
from pyspark.sql.functions import *
for i in df.columns[1:6]:
  data = df.agg({i : 'mean'}).first()[0]
  print(f"Mean value for {i} is {int(data)}")
  df = df.withColumn(i, when(df[i]==0, int(data)).otherwise(df[i]))

Mean value for Glucose is 121
Mean value for BloodPressure is 69
Mean value for SkinThickness is 20
Mean value for Insulin is 80
Mean value for BMI is 32


In [21]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|     80|25.6|                   0.294| 28|      0|


In [23]:
for col in df.columns:
  print(f"correlation to outcome for {col} is {(df.stat.corr('Outcome', col))}")

correlation to outcome for Pregnancies is 0.22443699263363961
correlation to outcome for Glucose is 0.48796646527321064
correlation to outcome for BloodPressure is 0.17171333286446713
correlation to outcome for SkinThickness is 0.1659010662889893
correlation to outcome for Insulin is 0.1711763270226193
correlation to outcome for BMI is 0.2827927569760082
correlation to outcome for DiabetesPedigreeFunction is 0.1554590791569403
correlation to outcome for Age is 0.23650924717620253
correlation to outcome for Outcome is 1.0


In [26]:
from pyspark.ml.feature import VectorAssembler
assembler  = VectorAssembler(inputCols = ['Pregnancies', 'Glucose', 'BloodPressure',
                                          'SkinThickness', 'Insulin',  'BMI'], outputCol = 'features')
output_data = assembler.transform(df)

In [28]:
output_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- features: vector (nullable = true)



In [29]:
output_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|[2.0,138.0,62.0,3...|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|[0.0,84.0,82.0,31...|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|[0.0,145.0,69.0,2...|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|[0.0,135.0,68.0,4...|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|[1.0,139.0,62.0,4...|
|          0|    173|           

In [30]:
from pyspark.ml.classification import LogisticRegression
final_data = output_data.select('features', 'Outcome')

In [32]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Outcome: integer (nullable = true)



In [33]:
train, test = final_data.randomSplit([0.7, 0.3])
models = LogisticRegression(labelCol = 'Outcome')
model = models.fit(train)

In [34]:
summary = model.summary

In [35]:
summary.predictions.describe().show()

+-------+-------------------+------------------+
|summary|            Outcome|        prediction|
+-------+-------------------+------------------+
|  count|               1441|              1441|
|   mean|0.34836918806384454|0.2734212352532963|
| stddev| 0.4766190736671152|0.4458699621325528|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



In [36]:
#Evaluating and testing our model

In [37]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = model.evaluate(test)


In [41]:
predictions.predictions.show(20)

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[0.0,57.0,60.0,20...|      0|[4.55294993168086...|[0.98957377374473...|       0.0|
|[0.0,67.0,76.0,20...|      0|[2.31699066084239...|[0.9102744565545,...|       0.0|
|[0.0,78.0,88.0,29...|      0|[2.47935470655384...|[0.92268177510787...|       0.0|
|[0.0,84.0,82.0,31...|      0|[2.21454238182551...|[0.90154784163275...|       0.0|
|[0.0,84.0,82.0,31...|      0|[2.21454238182551...|[0.90154784163275...|       0.0|
|[0.0,86.0,68.0,32...|      0|[2.39036626011649...|[0.91608972656661...|       0.0|
|[0.0,91.0,68.0,32...|      0|[1.92558610246818...|[0.87276006278705...|       0.0|
|[0.0,93.0,60.0,20...|      0|[2.19546185100778...|[0.89984124273330...|       0.0|
|[0.0,93.0,60.0,25...|      0|[2.70233570374976...|[0.93716432787281...|    

In [42]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='Outcome')
evaluator.evaluate(model.transform(test))

0.8442300405165123

In [43]:
model.save('model')

In [44]:
from pyspark.ml.classification import LogisticRegressionModel
model = LogisticRegressionModel.load('model')